In [44]:
# Network Algorithms Homework Project


In [55]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from geopy import distance

pd.set_option("display.max_rows", None, "display.max_columns", None)


In [46]:
# Obtain the data
airports = pd.read_csv('airports.csv')
routes = pd.read_csv('routes.csv')

In [47]:
def filter_na(df, column):
    na_rows = df[column].str.len() < 3
    return df[~na_rows]

# Filter the N/A values in the data
filtered_airports = filter_na(airports, "IATA")
filtered_routes = filter_na(routes, 'Source airport ID')
filtered_routes = filter_na(filtered_routes, 'Destination airport ID')

In [48]:
def get_countries_of(region):
    if (region == 'Europe'):
        import requests
        response = requests.get("https://restcountries.com/v3.1/region/europe")
        l = response.json()
        europe = []
        for i in range(len(l)):
            europe.append(l[i]['name']['common'])
        return europe
    # future support for other regions

def filter_to_europe(df, attr):
    europe = get_countries_of('Europe')
    return df[df[attr].isin(europe)]

# Choose only one region (europe in our example)
filtered_european_airports = filter_to_europe(filtered_airports, 'Country')

In [49]:
# Filter routes according to filtered airports
iatas = filtered_european_airports['IATA']
filtered_european_routes = filtered_routes[filtered_routes['Source airport'].isin(iatas) & filtered_routes['Destination airport'].isin(iatas)]
# print(filtered_european_routes)

In [50]:
def get_dists(routes, airports):
    dist = list()

    for ind in routes.index:

        try:
            source_id = routes['Source airport ID'][ind]
            destination_id = routes['Destination airport ID'][ind]

            source = airports.loc[airports['Airport ID'] == int(source_id)]
            destination = airports.loc[airports['Airport ID'] == int(destination_id)]

            if(
                len(source) == 0
            ):
                print('skipped the following source ID:')
                print(source_id)
                continue
            
            if(
                len(destination) == 0
            ):
                print('skipped the following destination ID:')
                print(destination_id)
                continue

            dist.append(distance.distance(
                (float(source['Latitude'].to_string().split(' ')[-1]), float(source['Longitude'].to_string().split(' ')[-1])),
                (float(destination['Latitude'].to_string().split(' ')[-1]), float(destination['Longitude'].to_string().split(' ')[-1]))
            ).km)
        except:
            print('----')
            print(source_id, destination_id)

    return dist

In [51]:
dist = get_dists(filtered_european_routes, filtered_european_airports)

In [66]:

# returns all the information about an airport by its IATA
def airport_info(data_airports, iata):
    return data_airports.loc[data_airports['IATA'] == iata]

# to display all the columns
# pd.set_option('display.max_columns', None)-

# array dist is an array of all the distances between sources and destinations that should be created and filled by Rustam
# once it's filled a new data frame with the Source, Destination and Distance values will be created

data = [filtered_european_routes['Source airport'],filtered_european_routes['Destination airport'], pd.Series(np.array(dist), name = 'Distance')]

# new DataFrame only with the Source, Destination and Distance values
df = pd.DataFrame(data).T
# print(df)

      Airline Airline ID Source airport Source airport ID Destination airport  \
0          2B        410            AER              2965                 KZN   
1          2B        410            ASF              2966                 KZN   
2          2B        410            ASF              2966                 MRV   
3          2B        410            CEK              2968                 KZN   
4          2B        410            CEK              2968                 OVB   
5          2B        410            DME              4029                 KZN   
6          2B        410            DME              4029                 NBC   
8          2B        410            DME              4029                 UUA   
9          2B        410            EGO              6156                 KGD   
10         2B        410            EGO              6156                 KZN   
12         2B        410            KGD              2952                 EGO   
13         2B        410    